### Segmentation according to the peaks of PPG

In [ ]:
def segmentation(pid, rid, winLenInSec_=2, srate=125, newSrate=125, normFlag=True):
    """
    Perform segmentation for a given recording designated by PID and RID
    @Parameters:
    pid: integer, PID, Part ID, under folder "/Dropbox/projects/Blood_Pressure/WFDB/database/KData/BloodPressureDataset".
    rid: integer, RID, Recording ID, under associated PID.
    winLenInSec: integer, window length that used for the segementation. The unit is second.
    srate: integer, sampling rate of the loaded signal
    newSrate: integer, required sampling rate after down sampling. If no down sampling is required, newSrate = srate.
    normFlag: boolean, request normalization or not.
    @Return:
    X: array, shape(num_sample, num_winLenInPnt), segmented PPG signal
    Y: array, shape(num_sample, num_winLenInPnt), segmented ABP signal
    """
    